### Getting the Mid values based on Bid and Ask values

In [1]:
if 'param' in locals():
    param
else:
#   param = <YOUR DATAFRAME>
    pass

In [2]:
if 'mode' in locals():
    mode
else:
#   mode = <YOUR MODE>
    pass

In [3]:
df_history2 = param.copy()
df_history2

NameError: name 'param' is not defined

In [ ]:
# Remove NaN Columns
df_history2 = df_history2.drop(df_history2.columns[[9,10,11,12]], axis=1)
df_history2

In [ ]:
pip install round2

In [ ]:
from round2 import round2

def averageTwoFloats(x,y):
    return round2((x+y)/2, 5)

In [ ]:
df_history2[('mid', 'Open')]  = df_history2.apply(lambda x: averageTwoFloats(x[('bid', 'Open')],  x[('ask', 'Open')]), axis=1)
df_history2[('mid', 'High')]  = df_history2.apply(lambda x: averageTwoFloats(x[('bid', 'High')],  x[('ask', 'High')]), axis=1)
df_history2[('mid', 'Low')]   = df_history2.apply(lambda x: averageTwoFloats(x[('bid', 'Low')],   x[('ask', 'Low')]), axis=1)
df_history2[('mid', 'Close')] = df_history2.apply(lambda x: averageTwoFloats(x[('bid', 'Close')], x[('ask', 'Close')]), axis=1)

df_history2

In [ ]:
df_history2 = df_history2.drop(df_history2.head(1).index)
df_history2

### Filter out those meaningful Columns

In [ ]:
df_trans = df_history2[[('DateTime', ''), ('mid', 'Open'), ('mid', 'High'), ('mid', 'Low'), ('mid', 'Close'), ('last', 'Volume')]].copy()
df_trans

In [ ]:
df_trans.columns = df_trans.columns.droplevel()
df_trans.columns = ['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume']
df_trans

### Feature Engineering

In [ ]:
df_trans3 = df_trans.copy()

In [ ]:
def diffTwoFloats(x,y):
    return round2(x-y, 5)

In [ ]:
def signIndicator(x):
    if x > 0:
        return -1
    elif x < 0:
        return 1
    else:
        return 0

In [ ]:
df_trans3['O-H'] = df_trans3.apply(lambda x: abs(diffTwoFloats(x['Open'], x['High'])), axis=1)
df_trans3['O-L'] = df_trans3.apply(lambda x: diffTwoFloats(x['Open'], x['Low']), axis=1)

df_trans3['C-H'] = df_trans3.apply(lambda x: abs(diffTwoFloats(x['Close'], x['High'])), axis=1)
df_trans3['C-L'] = df_trans3.apply(lambda x: diffTwoFloats(x['Close'], x['Low']), axis=1)

df_trans3['H-L'] = df_trans3.apply(lambda x: diffTwoFloats(x['High'], x['Low']), axis=1)

df_trans3['O-C-Value'] = df_trans3.apply(lambda x: abs(diffTwoFloats(x['Open'], x['Close'])), axis=1)
df_trans3['O-C-Sign'] = df_trans3.apply(lambda x: signIndicator(x['Open'] - x['Close']), axis=1)

df_trans3

### Tecnical Analysis

In [4]:
import ta

In [ ]:
df_trans3.describe()

### Normalization

In [ ]:
df_min_max_scaled = pd.DataFrame()

In [ ]:
# Doing the Min-Max Normalization
# normX = (X - minX) / (maxX - minX)

unNormalizeParams = {}

if mode == 'trainer':
    for column in df_trans3.columns:
        df_min_max_scaled[column] = (df_trans3[column] - df_trans3[column].min()) / (df_trans3[column].max() - df_trans3[column].min())
        unNormalizeParams[column] = {'min':df_trans3[column].min(), 'max':df_trans3[column].max()}
elif mode == 'predictor':
    import json
    f = open('../models/unnormalized_parameters.json')
    data = json.load(f)
    f.close()
    for column in df_trans3.columns:
        if column == 'DateTime':
            df_min_max_scaled[column] = (df_trans3[column] - datetime.strptime(data[column]['min'], "%Y-%m-%d %H:%M:%S")) / (datetime.strptime(data[column]['max'], "%Y-%m-%d %H:%M:%S") - datetime.strptime(data[column]['min'], "%Y-%m-%d %H:%M:%S"))
        else:
            df_min_max_scaled[column] = (df_trans3[column] - data[column]['min']) / (data[column]['max'] - data[column]['min'])
else:
    pass

In [ ]:
# To unNormalize the Min-Max Normalization
# X = normX * (maxX - minX) + minX

In [ ]:
unNormalizeParams

In [ ]:
df_min_max_scaled

In [ ]:
df_min_max_scaled.describe()

In [ ]:
df_min_max_scaled = df_min_max_scaled.drop(df_min_max_scaled.columns[[0]], axis=1)
df_min_max_scaled